# Tutorial for CLI tool `clkhash`

For this tutorial we are going to process a data set for private linkage with clkhash using the command line tool `clkutil`. Note you can also use the [Python API](./tutorial_api.ipynb).

The Python package `recordlinkage` has a [tutorial](http://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html) linking data sets in the clear, we will try duplicate that in a privacy preserving setting.

First install clkhash, recordlinkage and a few data science tools (pandas and numpy).

In [ ]:
!pip install -U clkhash recordlinkage numpy pandas

In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

## Data Exploration

First we have a look at the dataset.

In [3]:
dfA, dfB = load_febrl4()

dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


Note that for computing this linkage we will **not** use the social security id column or the `rec_id` index.

In [4]:
dfA.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2',
       'suburb', 'postcode', 'state', 'date_of_birth', 'soc_sec_id'],
      dtype='object')

In [5]:
dfA.to_csv('PII_a.csv')

## Hashing Schema Definition

A hashing schema instructs clkhash how to treat each column for generating CLKs. A detailed description of the hashing schema can be found in the [api docs](http://clkhash.readthedocs.io/en/latest/schema.html). We will ignore the columns 'rec_id' and 'soc_sec_id' for CLK generation.



In [6]:
%cat _static/febrl_schema_v2_overweight.json

{
  "version": 2,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
        "info": "c2NoZW1hX2V4YW1wbGU=",
        "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
        "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "rec_id",
      "ignored": true
    },
    {
      "identifier": "given_name",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 500}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "surname",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 500}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "street_number",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "strategy": {"numBits": 200}, "missingValue": {"sentinel": ""} }


## Validate the schema

The command line tool can check that the linkage schema is valid:

In [7]:
!clkutil validate-schema _static/febrl_schema_v2_overweight.json

schema is valid


## Hash the data

We can now hash our Personally Identifiable Information (PII) data from the CSV file using our defined linkage schema. We must provide two *secret keys* to this command - these keys have to be used by both parties hashing data. For this toy example we will use the keys _'key1'_ and _'key2'_, for real data, make sure that the keys contain enough entropy, as knowledge of these keys is sufficient to reconstruct the PII information from a CLK! Also, **do not share these keys with anyone, except the other participating party.**

In [8]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_overweight.json clks_a.json

generating CLKs: 100%|█| 5.00k/5.00k [00:00<00:00, 1.52kclk/s, mean=925, std=11.5]
CLK data written to clks_a.json


## Inspect the output

clkhash has hashed the PII, creating a Cryptographic Longterm Key for each entity. The stats output shows that the mean popcount (number of bits set) is quite high (925 out of 1024) which can effect accuracy.

To reduce the popcount you can modify the individual _'numBits'_ values for the different fields. It allows to tune the contribution of a column to the CLK. This can be used to de-emphasise columns which are less suitable for linkage (e.g. information that changes frequently).

First, we will reduce the value of *numBits* for each feature.

In [9]:
%cat _static/febrl_schema_v2_reduced.json

{
  "version": 2,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
        "info": "c2NoZW1hX2V4YW1wbGU=",
        "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
        "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "rec_id",
      "ignored": true
    },
    {
      "identifier": "given_name",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 100}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "surname",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 100}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "street_number",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "strategy": {"numBits": 100}, "missingValue": {"sentinel": ""} }


In [10]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_reduced.json clks_a.json

generating CLKs: 100%|█| 5.00k/5.00k [00:00<00:00, 7.46kclk/s, mean=633, std=14.1]
CLK data written to clks_a.json


And now we will modify the `numBits` values again, this time de-emphasising the contribution of the address related columns.

In [11]:
%cat _static/febrl_schema_v2_final.json

{
  "version": 2,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
        "info": "c2NoZW1hX2V4YW1wbGU=",
        "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
        "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "rec_id",
      "ignored": true
    },
    {
      "identifier": "given_name",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 200}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "surname",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 64 },
      "hashing": { "ngram": 2, "strategy": {"numBits": 150}, "hash": {"type": "doubleHash"} }
    },
    {
      "identifier": "street_number",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "strategy": {"numBits": 50}, "missingValue": {"sentinel": ""} }
 

In [12]:
!clkutil hash PII_a.csv key1 key2 _static/febrl_schema_v2_final.json clks_a.json

generating CLKs: 100%|█| 5.00k/5.00k [00:00<00:00, 7.90kclk/s, mean=543, std=13.1]
CLK data written to clks_a.json


Great, now approximately half the bits are set in each CLK. 

Each CLK is serialized in a JSON friendly base64 format:

In [13]:
# If you have jq tool installed:
#!jq .clks[0] clks_a.json

import json
json.load(open('clks_a.json'))['clks'][0]

'GnoZ2WzD14Mwib5NaXBL05xeaj6h8Hq06Xza8KcaHWWWrHygGXmNHuRW9tOd314KKvmnu5bzYELAIstOMPcPYEY97CjZJqrAg8S/CwL3PdHdtplV1ao2wJ+Ve53iXg4Vd988b3mhTsIQQd+7Xj2QXdpVSsSWNkmkX8uVKYI7nv0='

## Hash data set B

Now we hash the second dataset using the same keys and same schema.

In [14]:
dfB.to_csv('PII_b.csv')

!clkutil hash PII_b.csv key1 key2 _static/febrl_schema_v2_final.json clks_b.json

generating CLKs: 100%|█| 5.00k/5.00k [00:00<00:00, 8.08kclk/s, mean=542, std=15.5]
CLK data written to clks_b.json


## Find matches between the two sets of CLKs

We have generated two sets of CLKs which represent entity information in a privacy-preserving way. The more similar two CLKs are, the more likely it is that they represent the same entity.

For this task we will use the entity service, which is provided by Data61. 
The necessary steps are as follows:
- The analyst creates a new project with the output type 'mapping'. They will receive a set of credentials from the server.
- The analyst then distributes the `update_tokens` to the participating data providers.
- The data providers then individually upload their respective CLKs.
- The analyst can create *runs* with various thresholds (and other settings)
- After the entity service successfully computed the mapping, it can be accessed by providing the `result_token`

First we check the status of an entity service:

In [15]:
SERVER = 'https://testing.es.data61.xyz'

!clkutil status --server={SERVER}

{"project_count": 1226, "rate": 503702171, "status": "ok"}


The analyst creates a new project on the entity service by providing the hashing schema and result type. The server returns a set of credentials which provide access to the further steps for project.

In [16]:
!clkutil create-project --server={SERVER} --schema schema.json --output credentials.json --type "mapping" --name "tutorial"

Project created


The returned credentials contain a 
- `project_id`, which identifies the project
- `result_token`, which gives access to the mapping result, once computed
- `upload_tokens`, one for each provider, allows uploading CLKs.

In [17]:
credentials = json.load(open('credentials.json', 'rt'))
!python -m json.tool credentials.json

{
    "project_id": "9cb00ea7424692d75b3e2b06d87826b1ef5eaf707d8b710e",
    "result_token": "9b5b3eae23569ae200c87af765e87f951ce1bc6e30f27610",
    "update_tokens": [
        "9799e866dd51979d78ccae11b1741a777029e347dbf19d48",
        "c8518faea21267785d4e998a8060c242ba5a6a575aeaa32f"
    ]
}


### Uploading the CLKs to the entity service
Each party individually uploads its respective CLKs to the entity service. They need to provide the `resource_id`, which identifies the correct mapping, and an `update_token`.

In [18]:
!clkutil upload \
       --project="{credentials['project_id']}" \
        --apikey="{credentials['update_tokens'][0]}" \
        --output "upload_a.json" \
        --server="{SERVER}" \
       "clks_a.json"
    
!clkutil upload \
       --project="{credentials['project_id']}" \
        --apikey="{credentials['update_tokens'][1]}" \
        --output "upload_b.json" \
        --server="{SERVER}" \
       "clks_b.json"

Now that the CLK data has been uploaded the analyst can create one or more *runs*. Here we will start by calculating a mapping with a threshold of `0.9`:

In [19]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.9 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run A"

Entity Matching Server: https://testing.es.data61.xyz


In [20]:
run_info = json.load(open('run_info.json', 'rt'))
run_info

{'name': 'CLI tutorial run A',
 'notes': 'Run created by clkhash 0.12.2.dev0',
 'run_id': '2909a60505a0fd93c925c5e02088c22aa5a4731d03522ebc',
 'threshold': 0.9}

## Results

Now after some delay (depending on the size) we can fetch the results. This can be done with clkutil:

In [21]:
!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']
    mapping = {int(k): int(v) for k,v in str_mapping.items()}
print('The service linked {} entities.'.format(len(mapping)))

State: running
Stage (2/3): compute similarity scores
State: running
Stage (2/3): compute similarity scores
State: completed
Stage (3/3): compute output
Received result
The service linked 3392 entities.


Let's investigate some of those matches and the overall matching quality. In this case we have the ground truth so we can compute the precision, recall, and accuracy.

In [22]:
with open('PII_a.csv', 'rt') as f:
    a_raw = f.readlines()
with open('PII_b.csv', 'rt') as f:
    b_raw = f.readlines()

num_entities = len(b_raw) - 1

def describe_accuracy(mapping):
    print('idx_a, idx_b, rec_id_a, rec_id_b')
    print('--------------------------------')
    for a_i in range(10):
        if a_i in mapping:
            a_data = a_raw[a_i + 1].split(',')
            b_data = b_raw[mapping[a_i] + 1].split(',')
            print('{}, {}, {}, {}'.format(a_i+1, mapping[a_i]+1, a_data[0], b_data[0]))

    TP = 0; FP = 0; TN = 0; FN = 0
    for a_i in range(num_entities):
        if a_i in mapping:
            if a_raw[a_i + 1].split(',')[0].split('-')[1] == b_raw[mapping[a_i] + 1].split(',')[0].split('-')[1]:
                TP += 1
            else:
                FP += 1
                # as we only report one mapping for each element in PII_a, 
                # then a wrong mapping is not only a false positive, but 
                # also a false negative, as we won't report the true mapping.
                FN += 1 
        else:
            FN += 1 # every element in PII_a has a partner in PII_b

    print('--------------------------------')
    print('Precision: {}, Recall: {}, Accuracy: {}'.format(TP/(TP+FP), TP/(TP+FN), (TP+TN)/(TP+TN+FP+FN)))

In [23]:
describe_accuracy(mapping)

idx_a, idx_b, rec_id_a, rec_id_b
--------------------------------
2, 2751, rec-1016-org, rec-1016-dup-0
3, 4657, rec-4405-org, rec-4405-dup-0
4, 4120, rec-1288-org, rec-1288-dup-0
5, 3307, rec-3585-org, rec-3585-dup-0
6, 2306, rec-298-org, rec-298-dup-0
7, 3945, rec-1985-org, rec-1985-dup-0
8, 993, rec-2404-org, rec-2404-dup-0
9, 4613, rec-1473-org, rec-1473-dup-0
10, 3630, rec-453-org, rec-453-dup-0
--------------------------------
Precision: 1.0, Recall: 0.6784, Accuracy: 0.6784


Precision tells us about how many of the found matches are actual matches. The score of 1.0 means that we did perfectly in this respect, however, **recall**, the measure of how many of the actual matches were correctly identified, is quite low with only 67%.

Let's go back and create another mapping with a `threshold` value of `0.8`.

In [24]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.8 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run B"

run_info = json.load(open('run_info.json', 'rt'))

!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']
    mapping = {int(k): int(v) for k,v in str_mapping.items()}

print('The service linked {} entities.'.format(len(mapping)))
describe_accuracy(mapping)

Entity Matching Server: https://testing.es.data61.xyz
State: completed
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
Received result
The service linked 4409 entities.
idx_a, idx_b, rec_id_a, rec_id_b
--------------------------------
2, 2751, rec-1016-org, rec-1016-dup-0
3, 4657, rec-4405-org, rec-4405-dup-0
4, 4120, rec-1288-org, rec-1288-dup-0
5, 3307, rec-3585-org, rec-3585-dup-0
6, 2306, rec-298-org, rec-298-dup-0
7, 3945, rec-1985-org, rec-1985-dup-0
8, 993, rec-2404-org, rec-2404-dup-0
9, 4613, rec-1473-org, rec-1473-dup-0
10, 3630, rec-453-org, rec-453-dup-0
--------------------------------
Precision: 1.0, Recall: 0.8818, Accuracy: 0.8818


Great, for this threshold value we get a precision of 100% and a recall of 88%. 

The explanation is that when the information about an entity differs slightly in the two datasets (e.g. spelling errors, abbrevations, missing values, ...) then the corresponding CLKs will differ in some number of bits as well. For the datasets in this tutorial the perturbations are such that only 67% of the derived CLK pairs overlap more than 90% (the first threshold). Whereas 88% of all matching pairs overlap more than 80%.

If we keep reducing the threshold value, then we will start to observe mistakes in the found matches -- the precision decreases. But at the same time the recall value will keep increasing for a while, as a lower threshold allows for more of the actual matches to be found, e.g.: for threshold 0.7, we get precision: 99% and recall: 98%:

In [25]:
!clkutil create --verbose  \
    --server="{SERVER}" \
    --output "run_info.json" \
    --threshold=0.70 \
    --project="{credentials['project_id']}" \
    --apikey="{credentials['result_token']}" \
    --name="CLI tutorial run B"

run_info = json.load(open('run_info.json', 'rt'))

!clkutil results --watch \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --run="{run_info['run_id']}" \
        --server="{SERVER}" \
        --output results.txt

with open('results.txt') as f:
    str_mapping = json.load(f)['mapping']
    mapping = {int(k): int(v) for k,v in str_mapping.items()}

print('The service linked {} entities.'.format(len(mapping)))
describe_accuracy(mapping)

Entity Matching Server: https://testing.es.data61.xyz
State: running
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
Received result
The service linked 4943 entities.
idx_a, idx_b, rec_id_a, rec_id_b
--------------------------------
1, 1450, rec-1070-org, rec-1070-dup-0
2, 2751, rec-1016-org, rec-1016-dup-0
3, 4657, rec-4405-org, rec-4405-dup-0
4, 4120, rec-1288-org, rec-1288-dup-0
5, 3307, rec-3585-org, rec-3585-dup-0
6, 2306, rec-298-org, rec-298-dup-0
7, 3945, rec-1985-org, rec-1985-dup-0
8, 993, rec-2404-org, rec-2404-dup-0
9, 4613, rec-1473-org, rec-1473-dup-0
10, 3630, rec-453-org, rec-453-dup-0
--------------------------------
Precision: 0.9997976937082743, Recall: 0.9884, Accuracy: 0.9882023595280944


However, reducing the threshold further will eventually lead to a decrease in both precision and recall. Thus it is important to choose an appropriate threshold for the amount of perturbations present in the data.

Feel free to go back to the CLK generation and experiment on how different setting will affect the matching quality.

Finally to remove the uploaded CLK data from the service delete the project:

In [26]:
# TODO

#!clkutil delete-project --project="{credentials['project_id']}" \
#        --apikey="{credentials['result_token']}" \
#        --run="{run_info['run_id']}" \
#        --server="{SERVER}"